In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# A short interactive guide to brio

In this example we will construct SAILnet (Zylderberg et al, 2011), a simple network for performing sparse inference with local learning rules.

SAILnet consists of a single layer of leaky integrate and fire neurons that receive input from a stimulus (an image in this case) and send inhibition to each other. 

## brio architecture

All networks in brio are made out of two elements: Layers and Connections. 

A Layer holds state for a population of neurons. Layers can be excitatory, inhibitory or neither. The Layer class does not implement a specific neural model, giving significant wigggle room to subclasses. Implemented sublcasses include neural models for discrete LIF neurons and Boltzmann Machines.

Connection objects hold state for the weights connecting Layers. By design, Connection objects only have access to information about the Layers they connect, thus the learning rules implemented by Connection subclasses are necessarily local. 

Network objects hold pointers to the comprising Layers and Connections in addition to global utility methods to train the network and display progress plots.

Although brio is primarily an experimental platform and was designed to support arbitrary network topologies with SAILnet-like models, it has significant utility as a general purpose neural network library. 

## Instantiating networks in brio

To build a network in brio, first instantiate the different layers that are part of the network. 

For SAILnet we need two layers: an input layer whose only role is to hold the input stimuli, and a LIF layer. Let's create them:

In [ ]:
from brio.blocks import layer

# creates an InputLayer with 17**2 neurons
input_layer = layer.InputLayer(17**2, label='input layer')
# creates a LIFLayer with 20**2 neurons
## meta: I think need self connections flag enabled here to avoid run away excitation (by default False, should specify?)
lif_layer = layer.LIFLayer(20**2, label='lif layer')

The size of Layers must be specified when instantiated. Additionally several optional paramaters may be passed specifying a label, layer type (unconstrained (default), excitatory, inhibitory) or various flags. Subclasses may take additional parameters. Refer to the Layer docstring for details.

The next step is to specify the connectivity structure of the network by instantiating the required Connections. 

SAILnet has two connections: a feedforward connection from the input to the LIF population and a recurrent connection on the LIF population. The former updates its set of weights using Oja's rule whereas the latter uses Foldiak's rule.

In [ ]:
from brio.blocks import connection

ffwd_con = connection.OjaConnection(input_layer, lif_layer, label='ffwd connection')
recrnt_con = connection.FoldiakConnection(lif_layer, lif_layer, label='recrnt connection')

An input layer and output layer must be passed when instantiating connection objects. Optionally and additionally one may pass a label, a learning rate multiplier, and toggle to select a weight instantiation scheme. 

The last step to creating a network is to pass the layers to a Network object.

In [ ]:
from brio.blocks.network import Network
from brio.blocks.aux import NetworkParams

# Network.__init__ expects a list of Layer objects
layers = [ffwd_con, recrnt_con]

sailnet = Network(layers, params=NetworkParams())

There are two things to notice here:

Firstly, that I did not pass the connections to Network! This is not necessary as Connections add pointers to themselves to the Layers they connect when instantiated. Thus the network graph is stored in a decentralized fashion as a graph of pointers. 

Secondly, that I pass a NetworkParams object to Network. This object is nothing more than a dumb container class, designed to store global parameters. 

We can now begin training sailnet....